### Extracting branch flow information from the PV results obtained in the excel format

The branch flow results are obtained in PSSE for 2 studied cases
1. Without the Shunt at Bus 205  - (with the PV excel file - book_MVA.xlsx)
2. With the Shunt of 300 MVAR connected at Bus 205 - (with the PV excel file - book_shunt_MVA.xlsx)

Following is the python code used to obtain the PV branch flow results

In [1]:
import pandas as pd
import math
import altair as alt

Following function is defined to extract the branch flow information corresponding to the maximum incremental transfer from the PV results obtained in the excel format

In [2]:
def function_flow(book_name, sheet):
    """
    inputs: book_name, sheet
            book_name -> the obtained PV Excel file (.xlsx file type)
            sheet -> sheet name in the Excel file ('string')
    output: dataframe -> dataframe corresponind to the load (DataFrame)
    """
    data = pd.read_excel(book_name, sheet_name=sheet, header=1)
    data_bus = data.iloc[:,:6]
    data_bus[[data_bus.columns[0], data_bus.columns[1], data_bus.columns[2],data_bus.columns[3], data_bus.columns[4], data_bus.columns[5]]] = data_bus[[data_bus.columns[0], data_bus.columns[1], data_bus.columns[2], data_bus.columns[3], data_bus.columns[4], data_bus.columns[5]]].astype(str)
    data_bus['to'] = ' - '
    data['Bus Flow'] = data_bus['MW TRANSFER->']+ ' ' + data_bus['Unnamed: 1'] + data_bus['to']  + data_bus['Unnamed: 3'] +  ' ' + data_bus['Unnamed: 4']
    data_power = data.iloc[:,-2:]
    data_power['Load PF'] = 'PF = ' + str(round(math.cos((math.atan(float(volt_sheet.split(' ')[0])/20))),2))
    Max_power = data_power.columns[0]
    data_power['Maximum Incremental Transfer'] = Max_power
    data_power = data_power.rename(columns={data_power.columns[0]: "Branch Flow"})
    return(data_power)

The sheetnames of the output excel sheet is in the format of '39.69 Branch Flow', where 39.69 is the initial reactive part of the load considered for the analysis. The results are extracted by getting the sheet name and using the function defined above 

In [3]:
def function_MIT_all(book_name):
    loads = [39.69,34.64,30.36,26.66,23.38,20.40,17.63,15,12.39,10.79,9.68,0]
    volt_sheets = []
    for volt in loads:
        sheet = str(volt) + ' Branch Flow'
        volt_sheets.append(sheet)  
    volt_df_list = []
    for volt_sheet in volt_sheets:
        df = function_flow(book_name, volt_sheet) # the book_name used here 
        volt_df_list.append(df)
    df_flow = pd.concat(volt_df_list).reset_index(drop=True)
    return(df_flow)

1. Without the Shunt at Bus 205 - (with the PV excel file - book_MVA.xlsx)

In [4]:
loads = [39.69,34.64,30.36,26.66,23.38,20.40,17.63,15,12.39,10.79,9.68,0]
volt_sheets = []
for volt in loads:
    sheet = str(volt) + ' Branch Flow'
    volt_sheets.append(sheet)  
volt_df_list = []
for volt_sheet in volt_sheets:
    df = function_flow('book_MVA.xlsx', volt_sheet) # the book_name used here 
    volt_df_list.append(df)
df_flow_no = pd.concat(volt_df_list).reset_index(drop=True)

2. With the Shunt at Bus 205 - (with the PV excel file - book__shunt_MVA.xlsx)

In [5]:
loads = [39.69,34.64,30.36,26.66,23.38,20.40,17.63,15,12.39,10.79,9.68,0]
volt_sheets = []
for volt in loads:
    sheet = str(volt) + ' Branch Flow'
    volt_sheets.append(sheet)  
volt_df_list = []
for volt_sheet in volt_sheets:
    df = function_flow('book_shunt_MVA.xlsx', volt_sheet) # the book_name used here 
    volt_df_list.append(df)
df_flow = pd.concat(volt_df_list).reset_index(drop=True)

df_flow and df_flow_no are the dataframes obtained corresponding to the 2 studied cases. list_BF gives all the branch flows observed for the studied cases 

In [6]:
list_BF = list(df_flow['Bus Flow'].unique())

Branch flow in each branch is plotted as below for maximum incremental transfer

In [7]:
list_BF = sorted(df_flow['Bus Flow'].unique())
dropdown = alt.binding_select(name='Bus Flow ', options=list_BF)
select_BF = alt.selection_single(fields=['Bus Flow'], bind=dropdown)

chart_line = alt.Chart(df_flow, title = alt.TitleParams('Change the dropdown to show desired Branch Flow',subtitle = 'Hover over the plot to see Branch Flow value' )).mark_line(color='green').encode(
     alt.X('Load PF', title ='Load PF'),
     alt.Y('Branch Flow',title = 'Branch Flow'),
     alt.Color('Bus Flow')).properties(width = 600, height=700)

chart_point = alt.Chart(df_flow, title = 'test').mark_point(color='black').encode(
    alt.X('Load PF', title ='Load PF'),
    alt.Y('Branch Flow',title = 'Branch Flow')).properties(width = 600, height=700)

chart_p = chart_point+ chart_point.mark_text(align='left',dx=15, dy=8, color='black').encode(text='Maximum Incremental Transfer', tooltip = ['Load PF', 'Branch Flow'])
chart = chart_line+chart_p
chart.add_selection(select_BF).encode(
    opacity=alt.condition(select_BF, alt.value(0.7), alt.value(0.05)))

<ipython-input-7-2ab680c2c812>:3: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use selection_point instead.
  select_BF = alt.selection_single(fields=['Bus Flow'], bind=dropdown)
<ipython-input-7-2ab680c2c812>:16: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  chart.add_selection(select_BF).encode(


alt.LayerChart(...)

In [8]:
list_BF_no = sorted(df_flow_no['Bus Flow'].unique())
dropdown = alt.binding_select(name='Bus Flow ', options=list_BF_no)
select_BF_no = alt.selection_single(fields=['Bus Flow'], bind=dropdown)


chart_line_no = alt.Chart(df_flow_no, title = alt.TitleParams('Change the dropdown to show desired Branch Flow',subtitle = 'Hover over the plot to see Branch Flow value' )).mark_line(color='green').encode(
     alt.X('Load PF', title ='Load PF'),
     alt.Y('Branch Flow',title = 'Branch Flow'),
     alt.Color('Bus Flow')).properties(width = 600, height=700)

chart_point_no = alt.Chart(df_flow_no, title = 'test').mark_point(color='black').encode(
    alt.X('Load PF', title ='Load PF'),
    alt.Y('Branch Flow',title = 'Branch Flow')).properties(width = 600, height=700)

chart_p_no = chart_point_no + chart_point_no.mark_text(align='left',dx=15, dy=8, color='black').encode(text='Maximum Incremental Transfer', tooltip = ['Load PF', 'Branch Flow'])
chart_no = chart_line_no + chart_p_no
chart_no.add_selection(select_BF).encode(
    opacity=alt.condition(select_BF, alt.value(0.7), alt.value(0.05)))

<ipython-input-8-ea9c6df5acad>:3: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use selection_point instead.
  select_BF_no = alt.selection_single(fields=['Bus Flow'], bind=dropdown)
<ipython-input-8-ea9c6df5acad>:17: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  chart_no.add_selection(select_BF).encode(


alt.LayerChart(...)

For a desired incremental transfer chosen, the branch flow is plotted for each branch. To extract the desired flow through each branch the following 2 functions are defined

In [9]:
def function_flow_desired(book_name, sheet, desired_incremental_transfer):
    import numpy as np
    """
    inputs: book_name, sheet
            book_name -> the obtained PV Excel file (.xlsx file type)
            sheet -> sheet name in the Excel file ('string')
            desired_incremental_transfer -> desired incremental transfer in MW (int)
    output: dataframe -> dataframe corresponind to the load (DataFrame)
    """
    data = pd.read_excel(book_name, sheet_name=sheet, header=1)
    data_bus = data.iloc[:,:6]
    data_bus[[data_bus.columns[0], data_bus.columns[1], data_bus.columns[2],data_bus.columns[3], data_bus.columns[4], data_bus.columns[5]]] = data_bus[[data_bus.columns[0], data_bus.columns[1], data_bus.columns[2], data_bus.columns[3], data_bus.columns[4], data_bus.columns[5]]].astype(str)
    data_bus['to'] = ' - '
    data['Branch Flow'] = data_bus['MW TRANSFER->']+ ' ' + data_bus['Unnamed: 1'] + data_bus['to']  + data_bus['Unnamed: 3'] +  ' ' + data_bus['Unnamed: 4']
    data_power = data
    if desired_incremental_transfer in data_power:
        data_power = data[['Branch Flow',desired_incremental_transfer]] 
        col = list(data_power.columns)
        desired_incremental_transfer = col[1]
        data_power = data_power[['Branch Flow',desired_incremental_transfer]]
        new_bus_name = 'Branch Flow for incremental transfer of '+ str(desired_incremental_transfer) + ' MW'
        data_power = data_power.rename(columns={data_power.columns[1]:new_bus_name})
    else:
        data_power[desired_incremental_transfer] = np.nan
        data_power = data_power[['Branch Flow',desired_incremental_transfer]]
        new_bus_name = 'Branch Flow for incremental transfer of '+ str(desired_incremental_transfer) + ' MW'
        data_power = data_power.rename(columns={data_power.columns[1]:new_bus_name})
    return data_power

In [10]:
def function_flow_all(book_name, desired_flow):
    loads = [39.69,34.64,30.36,26.66,23.38,20.40,17.63,15,12.39,10.79,9.68,0]
    volt_sheets = []
    for volt in loads:
        sheet = str(volt) + ' Branch Flow'
        volt_sheets.append(sheet)  
    volt_df_list = []
    for volt_sheet in volt_sheets:
        df = function_flow_desired(book_name, volt_sheet, desired_flow)
        df['Load PF'] = 'PF = ' + str(round(math.cos((math.atan(float(volt_sheet.split(' ')[0])/20))),2))# the book_name used here 
        volt_df_list.append(df)
    df_flow= pd.concat(volt_df_list).reset_index(drop=True)
    return df_flow

Using the defined functions, branch flow is calculated for a desired tranfer of 600 MW. The results are plotted as below. For lower power factors, for the same incremental transfers, the branch flow is lower for case where reactive power support is available for branches in Area 2. That means utilization of transmission line capacity is better. For interarea tranfers, the flow with and without the shunt was interchangeable. 

In [11]:
df_600 = function_flow_all('book_MVA.xlsx',600)
df_600 = df_600.rename(columns={'Load PF':'load_PF'})

In [12]:
df_600_1 = function_flow_all('book_shunt_MVA.xlsx',600)
df_600_1 = df_600_1.rename(columns={'Load PF':'load_PF'})

In [13]:
for each_BF in list_BF:
    last_val = df_600['load_PF'].iloc[-1]
    df_filter = df_600[df_600['Branch Flow'] == each_BF]
    df_filter_1 = df_600_1[df_600_1['Branch Flow'] == each_BF]
    chart_line = alt.Chart(df_filter, title = each_BF).mark_line(color='blue').encode(
        alt.X('load_PF', title ='Load PF'),
        alt.Y(df_filter.columns[1],title = 'Branch Flow')).properties(width = 600, height=300)
    
    chart1 = chart_line + chart_line.mark_text(align='left', dx=15, dy=-1, color = 'blue').encode(text=alt.condition(alt.datum.load_PF == last_val,
                            alt.value('Without the shunt connected at Bus 205'),
                            alt.value('')))
    
    chart_line_1 = alt.Chart(df_filter_1, title = each_BF).mark_line(color='green').encode(
        alt.X('load_PF', title ='Load PF'),
        alt.Y(df_filter_1.columns[1],title = 'Branch Flow')).properties(width = 600, height=300)
    
    chart2 = chart_line_1 + chart_line_1.mark_text(align='left', dx=15, dy=20, color = 'green').encode(text=alt.condition(alt.datum.load_PF == last_val,
                            alt.value('With the shunt of 300 MVAR connected at Bus 205'),
                            alt.value('')))

    display(chart1 + chart2)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

...

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

### Conclusions

In the above analysis, the branch flow is extracted from PSSE PV analysis Excel sheet outputs and variation in branch flow is observed when bus 205 is connected with a shunt and when it is not connected with a shunt. It was observed that for area branch flows, for the same maximum incremental transfer, lower branch flow is needed when shunt is connected at the load bus.